# LLM Exercise with RAG

This notebook demonstrates how to use a language model with a retrieval-augmented generation (RAG) approach. The goal is to retrieve relevant documents from a knowledge base and use them to generate a response to a user query.

## Import Libraries

In [6]:
# Import Libraries to pull a model from Hugging Face and set it up with RAG
import os
import sys
import json
import torch
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from transformers import RagConfig, RagTokenForGeneration
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# Import libraries for efficient small LLMs
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# For RAG components
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader

# For file handling
import os
import glob
from tqdm.notebook import tqdm

## Small LLMs for Local Inference

For running inference locally with reasonable performance, there are several excellent small LLMs to consider:

1. **Phi-2** (2.7B parameters) - Microsoft's model with excellent reasoning for its size
2. **TinyLlama** (1.1B parameters) - Extremely compact with decent performance
3. **Mistral-7B-Instruct** - Great performance/size trade-off with 7B parameters
4. **Llama-2-7B** - Good general-purpose model that can be quantized further

These models can be loaded with quantization (4-bit or 8-bit) to reduce memory requirements while maintaining most of the performance.

## Load a Small LLM for Local Inference

Below are examples of how to load different small LLMs with quantization for efficient inference.

In [14]:
# Choose and load your preferred model
# Uncomment the model you want to use

# Option 1: Phi-2 (2.7B) - Best performance/size ratio
# model, tokenizer = load_phi2_model()
# print("Phi-2 model loaded successfully!")

# Modified function to load TinyLlama without GPU requirements
def load_tinyllama_cpu():
    """Load TinyLlama (1.1B parameters) for CPU inference without quantization"""
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="cpu",
        torch_dtype=torch.float32,  # Use regular precision instead of quantization
    )
    return model, tokenizer

# Option 2: TinyLlama (1.1B) - Smallest size
model, tokenizer = load_tinyllama_cpu()
print("TinyLlama model loaded successfully (CPU mode)!")

# Option 3: Mistral (7B) - Best overall performance
# model, tokenizer = load_mistral_model()
# print("Mistral-7B model loaded successfully!")

# Create a text generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


TinyLlama model loaded successfully (CPU mode)!


## Set Up RAG with Local Documents

Now we'll set up a Retrieval-Augmented Generation (RAG) system using your local PDF text files.
This will allow the model to generate answers based on your document collection.

In [15]:
# Load text files from the data directory
def load_documents(directory="data"):
    """Load all text files from the specified directory"""
    txt_files = glob.glob(os.path.join(directory, "*.txt"))
    print(f"Found {len(txt_files)} text files in {directory}")
    
    documents = []
    for file_path in tqdm(txt_files, desc="Loading documents"):
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()
            documents.append({"content": content, "source": os.path.basename(file_path)})
    
    return documents

# Load and preprocess documents
documents = load_documents()
print(f"Loaded {len(documents)} documents")

Found 26 text files in data


Loading documents:   0%|          | 0/26 [00:00<?, ?it/s]

Loaded 26 documents


In [16]:
# Split documents into chunks for processing
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into manageable chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    chunks = []
    for doc in tqdm(documents, desc="Splitting documents"):
        doc_chunks = text_splitter.split_text(doc["content"])
        for chunk in doc_chunks:
            chunks.append({"content": chunk, "source": doc["source"]})
    
    return chunks

# Split documents into manageable chunks
chunks = split_documents(documents)
print(f"Created {len(chunks)} chunks from {len(documents)} documents")

Splitting documents:   0%|          | 0/26 [00:00<?, ?it/s]

Created 17312 chunks from 26 documents


In [17]:
# Load a small embedding model
def setup_embeddings_and_vectorstore(chunks):
    """Set up embeddings and vector storage for document retrieval"""
    # Use a small, efficient embedding model
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    
    # Extract chunk texts and their metadata
    texts = [chunk["content"] for chunk in chunks]
    metadatas = [{"source": chunk["source"]} for chunk in chunks]
    
    # Create FAISS vector store
    vectorstore = FAISS.from_texts(
        texts=texts,
        embedding=embeddings,
        metadatas=metadatas
    )
    
    return vectorstore

# Set up vector store for retrieval
vectorstore = setup_embeddings_and_vectorstore(chunks)
print("Vector store created successfully!")

C:\Users\Training Machine\AppData\Local\Temp\ipykernel_9976\1429758578.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created successfully!


## RAG Inference with Local LLM

Now we can use our local LLM with the RAG setup to answer questions based on the document collection.

In [18]:
def rag_query(query, num_chunks=3, generator=generator):
    """Query the RAG system with a user question"""
    # Retrieve relevant chunks
    retrieved_docs = vectorstore.similarity_search(query, k=num_chunks)
    retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
    sources = list(set([doc.metadata["source"] for doc in retrieved_docs]))
    
    # Create context-enriched prompt
    prompt = f"""Context information is below.
---------------------
{retrieved_text}
---------------------
Given the context information and no prior knowledge, answer the following question: {query}
"""
    
    # Generate response using the local LLM
    response = generator(prompt, max_new_tokens=512)[0]["generated_text"]
    
    # Remove the prompt from the response
    response = response.replace(prompt, "")
    
    return {
        "response": response,
        "sources": sources
    }

In [19]:
# Example query
query = "How do I set up a Cisco ASA 5505 firewall?"
result = rag_query(query)

print("Question:", query)
print("\nAnswer:")
print(result["response"])
print("\nSources:")
for source in result["sources"]:
    print(f"- {source}")

c:\Users\Training Machine\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Training Machine\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Question: How do I set up a Cisco ASA 5505 firewall?

Answer:

Answer: To set up a Cisco ASA 5505 firewall, follow these steps:

1. Download and install the latest version of the Cisco ASA Firewall Manager software from the Cisco website.

2. Open the ASA Firewall Manager console by clicking on the "Start" button and selecting "Run."

3. In the console, navigate to the "Administration" tab and click on "Add/Remove Devices."

4. Select "Add Device" and enter the details for the new device, including its name, IP address, and username or password.

5. Click "OK" to add the device to the list of devices.

6. Once the device has been added, click on it to open the device configuration screen.

7. Under the "General" tab, specify the name of the device, such as "MyDevice," and enter any necessary settings, such as the IP address and subnet mask.

8. For the "Security" tab, select the appropriate security policy for the device, such as "Basic" or "Advanced."

9. Click "Save" to save the chan

## Interactive RAG Query Interface

Use the cell below to query your document collection interactively.

In [ ]:
from ipywidgets import widgets
from IPython.display import display, HTML

# Create input widget
query_input = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Query:',
    layout=widgets.Layout(width='80%')
)

# Create output widget
output = widgets.Output()

# Define submit button callback
def on_submit_clicked(b):
    with output:
        output.clear_output()
        print("Processing query...")
        result = rag_query(query_input.value)
        print("\n\nAnswer:")
        print(result["response"])
        print("\nSources:")
        for source in result["sources"]:
            print(f"- {source}")

# Create and configure submit button
submit_button = widgets.Button(
    description='Submit',
    button_style='primary'
)
submit_button.on_click(on_submit_clicked)

# Display the interface
display(query_input, submit_button, output)

Text(value='', description='Query:', layout=Layout(width='80%'), placeholder='Type your question here...')

Button(button_style='primary', description='Submit', style=ButtonStyle())

Output()